In [58]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import time
import gc
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from collections import Counter


from utils import *

In [59]:
data_orig = pd.read_csv("train_data_sila/train_data.csv").set_index('index')
data_orig['Точка отказа'] = data_orig['Точка отказа'].replace("Диск ", "Диск")
data_orig = data_orig[data_orig['Точка отказа'] != 'Консультация']
data_orig['full_text'] = data_orig[['Тема', 'Описание']].apply(lambda x: x[0] + '\n' + x[1], axis=1)
data_train_orig, data_val_orig = train_test_split(data_orig, test_size=0.2, random_state=42)


data = pd.read_csv("data/synthetic_dataset_for_problem_type_hard.csv").iloc[:, 1:]
data['target'] = data['target'].replace("Диск ", "Диск")
data_train = data.rename(columns={'result': 'full_text', 'target': 'Точка отказа'})
data_train = data_train[data_train['Точка отказа'] != 'Консультация']

In [60]:
data_train.shape

(4810, 2)

In [64]:
model_id = "cointegrated/LaBSE-en-ru"
# model_id = "ai-forever/ruBert-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id)

# Data

In [67]:
def softmax(tensor, t=1.0):
    ex = torch.exp(tensor/t)
    ex_sum = torch.sum(ex, axis=0)
    return ex / ex_sum

In [68]:
data

,result,target
0,"На сервере возникли проблемы, не загружается с...",Материнская плата
1,"На сервере перестал запускаться, подозреваю, ч...",Материнская плата
2,На сервере возникли проблемы с материнской пла...,Материнская плата
3,Добрый день! На сервере перестала загружаться ...,Материнская плата
4,На сервере Сила возникли проблемы с материнско...,Материнская плата
...,...,...
4995,Здравствуйте! На ноутбуке Сила перестала работ...,Камера
4996,На ноутбуке Сила перестала работать камера. Пр...,Камера
4997,На ноутбуке Сила перестала работать камера. Пр...,Камера
4998,На ноутбуке Сила перестала работать камера. Пр...,Камера


In [69]:
def get_text_embedding(text):
    text_encoded = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors='pt')
    
    with torch.no_grad():
        # emb = torch.nn.functional.normalize(torch.mean(model(**text_encoded)[0], dim=1))
        emb = torch.nn.functional.normalize(model(**text_encoded)[1])
    
    return emb

In [70]:
from torch.utils.data import Dataset, DataLoader

class ContrastiveDataset(Dataset):
    def __init__(self, data, column, temperature_sampling=0.15):
        data = data.copy()

        label_counts = data[column].value_counts()
        single_labels = label_counts[label_counts == 1].index

        data = data[~data[column].isin(single_labels)]


        data = data.reset_index(drop=True)

        texts = data['full_text']
        labels = data[column]
        self.label_to_positive_ids = {label: data.index[data[column] == label].tolist() for label in data[column].unique()}
        self.label_to_negative_ids = {label: data.index[data[column] != label].tolist() for label in data[column].unique()}

        # self.sims = get_texts_similarities(texts)

        self.texts = texts
        self.labels = labels
        self.temperature_sampling = temperature_sampling

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text, label = self.texts[idx], self.labels[idx]
        positive_ids = self.label_to_positive_ids[label]
        negative_ids = self.label_to_negative_ids[label]


        positive_idx = np.random.choice(positive_ids)
        negative_idx = np.random.choice(negative_ids)

        negative_label = self.labels[negative_idx]

        return (text, self.texts[positive_idx], self.texts[negative_idx]), (label, negative_label)



class EvalDataset(Dataset):
    def __init__(self, data, column, temperature_sampling=0.15):

        data = data.reset_index(drop=True)

        self.texts = data['full_text']
        self.labels = data[column]

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

target_column = 'Точка отказа'
dataset_train = ContrastiveDataset(data_train, target_column)
dataset_train_for_eval = EvalDataset(data_train, target_column)

dataset_train_orig = ContrastiveDataset(data_train_orig, target_column)
dataset_train_orig_for_eval = EvalDataset(data_train_orig, target_column)

dataset_val = EvalDataset(data_val_orig, target_column)

In [72]:
class DataCollator:
    def __init__(self, tokenizer, stage='train'):
        self.tokenizer = tokenizer
        self.stage = stage

    def tokenize(self, text):
        return self.tokenizer(text, padding=True, truncation=True, max_length=512)['input_ids']

    def padding(self, encodings):
        max_length = max(len(encoding) for encoding in encodings)
        attention_masks = torch.stack([torch.tensor([1] * len(enc) + [0] * (max_length - len(enc))) for enc in encodings])
        encodings = torch.stack([torch.tensor(enc + [0] * (max_length - len(enc))) for enc in encodings])
        # encodings = [torch.cat(enc + torch.tensor([0] * (max_length - len(enc))), dim=0) for enc in encodings]

        return {'input_ids': encodings, 'attention_mask': attention_masks}

    def __call__(self, batch_unprocessed):

        if self.stage == 'train':
            batch = {'anchors' : [], 'positives': [], 'negatives' : []}
    
            for (anchor, pos, neg), (label, neg_label) in batch_unprocessed:
                batch['anchors'].append(self.tokenize(anchor))
                batch['positives'].append(self.tokenize(pos))
                batch['negatives'].append(self.tokenize(neg))
    
            batch = {k : self.padding(v) for k, v in batch.items()}
            
        elif self.stage == 'val':
            
            batch = {'encodings': [], 'labels': []}
    
            for text, label in batch_unprocessed:
                batch['encodings'].append(self.tokenize(text))
                batch['labels'].append(label)
    
            batch['encodings'] = self.padding(batch['encodings'])

        else:
            raise ValueError()
        
        return batch

data_collator_train = DataCollator(tokenizer=tokenizer, stage='train')
data_collator_val = DataCollator(tokenizer=tokenizer, stage='val')

In [73]:
dataloader_train = DataLoader(dataset_train, batch_size=8, collate_fn=data_collator_train, shuffle=True)
dataloader_train_for_eval = DataLoader(dataset_train_for_eval, batch_size=32, collate_fn=data_collator_val, shuffle=False)

dataloader_train_orig = DataLoader(dataset_train_orig, batch_size=8, collate_fn=data_collator_train, shuffle=True)
dataloader_train_for_eval_orig = DataLoader(dataset_train_orig_for_eval, batch_size=32, collate_fn=data_collator_val, shuffle=False)

dataloader_val = DataLoader(dataset_val, batch_size=32, collate_fn=data_collator_val, shuffle=False)

# Model

In [75]:
import torch.nn as nn
from torch import optim
from collections import defaultdict
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings("ignore")

In [76]:
class TripletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(TripletLoss, self).__init__()
        self.margin = margin
        
    def calc_euclidean(self, x1, x2):
        return (x1 - x2).pow(2).sum(1)
    
    def forward(self, anchor: torch.Tensor, positive: torch.Tensor, negative: torch.Tensor) -> torch.Tensor:
        distance_positive = self.calc_euclidean(anchor, positive)
        distance_negative = self.calc_euclidean(anchor, negative)
        losses = torch.relu(distance_positive - distance_negative + self.margin)

        return losses.mean()

criterion = TripletLoss(margin=1.0)

In [77]:
class Model(nn.Module):
    
    def __init__(self, model_id):
        super(Model, self).__init__()
        self.embedder = AutoModel.from_pretrained(model_id)

    def get_embed(self, text_encoded):
        # return torch.nn.functional.normalize(torch.mean(self.embedder(**text_encoded)[0], dim=1))
        return torch.nn.functional.normalize(self.embedder(**text_encoded)[1])
        
    def forward(self, anchors, positives=None, negatives=None):
        
        anchors_embeds = self.get_embed(anchors)
        if positives is None:
            return anchors_embeds
        positives_embeds = self.get_embed(positives)
        negatives_embeds = self.get_embed(negatives)
        
        return anchors_embeds, positives_embeds, negatives_embeds

In [78]:
for b in dataloader_train:
    break

# Training

In [80]:
empty_cache()

seed_everything(42)

model = Model(model_id)

optimizer = optim.Adam(model.parameters(), lr=1e-5)

num_epochs=1
device='cuda'

model.to(device)

Model(
  (embedder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(55083, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

In [81]:
num_samples_to_evaluate = 500

In [82]:
def process_batch(batch, device):
    for k in batch.keys():
            for data_type in batch[k].keys():
                batch[k][data_type] = batch[k][data_type].to(device)
    
model.train()
for epoch in tqdm(range(num_epochs), desc="Epochs"):
    running_loss = []
    for batch_idx, batch in tqdm(enumerate(dataloader_train), desc="Training", total=len(dataloader_train)):
        process_batch(batch, device)
        
        optimizer.zero_grad()
        anchors_embeds, positives_embeds, negatives_embeds = model(**batch)
        
        loss = criterion(anchors_embeds, positives_embeds, negatives_embeds)
        loss.backward()
        optimizer.step()
        
        running_loss.append(loss.cpu().detach().numpy())
    print("Epoch: {}/{} - Training Loss: {:.4f}".format(epoch+1, num_epochs, np.mean(running_loss)))

    all_labels_train = []
    all_embeddings_train = []
    
    for batch in dataloader_train_for_eval:
        batch['encodings'] = {k : v.to(device) for k, v in batch['encodings'].items()}
    
        with torch.no_grad():
            embeds = model(batch['encodings'])
    
        all_embeddings_train.append(embeds)
        all_labels_train.extend(batch['labels'])
    
    all_embeddings_train = torch.cat(all_embeddings_train, dim=0)
    
    train_label_to_ids = defaultdict(list)
    for i, label in enumerate(all_labels_train):
        train_label_to_ids[label].append(i)
    
    all_labels_val = []
    all_embeddings_val = []
    for batch in dataloader_val:
        batch['encodings'] = {k : v.to(device) for k, v in batch['encodings'].items()}
    
        with torch.no_grad():
            embeds = model(batch['encodings'])
    
        all_embeddings_val.append(embeds)
        all_labels_val.extend(batch['labels'])
    
    all_embeddings_val = torch.cat(all_embeddings_val, dim=0)
    
    predicted = []
    for embed in all_embeddings_val:
        dists = criterion.calc_euclidean(embed, all_embeddings_train)
        min_label_dict = float('inf')
        pred = -1
        for label in train_label_to_ids.keys():
            ids = np.random.choice(train_label_to_ids[label], min(len(train_label_to_ids[label]), num_samples_to_evaluate), replace=False)
            label_dist = dists[ids].mean()
            if label_dist < min_label_dict:
                min_label_dict = label_dist
                pred = label
    
        predicted.append(pred)
    
    metric = f1_score(all_labels_val, predicted, average='weighted', zero_division=0.0)
    print(f"F1 weighted: {metric}")
    
    print(classification_report(all_labels_val, predicted))
    print('='*50)

Training: 100%|██████████████████████████████| 602/602 [01:07<00:00,  8.91it/s]


Epoch: 1/1 - Training Loss: 0.0264


Epochs: 100%|████████████████████████████████████| 1/1 [01:12<00:00, 72.85s/it]

F1 weighted: 0.3263683851919146
                         precision    recall  f1-score   support

                   Jack       0.00      0.00      0.00         0
           Wi-fi модуль       0.00      0.00      0.00         1
            Аккумулятор       0.00      0.00      0.00         0
           Блок питания       1.00      0.17      0.29        12
             Вентилятор       0.00      0.00      0.00         0
               Динамики       0.00      0.00      0.00         0
                   Диск       0.50      0.50      0.50         4
                 Камера       0.00      0.00      0.00         1
             Клавиатура       0.00      0.00      0.00         0
                 Корпус       0.00      0.00      0.00         1
      Материнская плата       0.00      0.00      0.00         7
                Матрица       0.67      0.75      0.71         8
     Оперативная память       0.00      0.00      0.00         0
Программное обеспечение       0.00      0.00      0.00   

In [83]:
num_epochs = 8
best_metric = -1
    
model.train()
for epoch in tqdm(range(num_epochs), desc="Epochs"):
    running_loss = []
    for batch_idx, batch in tqdm(enumerate(dataloader_train_orig), desc="Training", total=len(dataloader_train_orig)):
        process_batch(batch, device)
        
        optimizer.zero_grad()
        anchors_embeds, positives_embeds, negatives_embeds = model(**batch)
        
        loss = criterion(anchors_embeds, positives_embeds, negatives_embeds)
        loss.backward()
        optimizer.step()
        
        running_loss.append(loss.cpu().detach().numpy())
    print("Epoch: {}/{} - Training Loss: {:.4f}".format(epoch+1, num_epochs, np.mean(running_loss)))

    all_labels_train = []
    all_embeddings_train = []
    
    for batch in dataloader_train_for_eval_orig:
        batch['encodings'] = {k : v.to(device) for k, v in batch['encodings'].items()}
    
        with torch.no_grad():
            embeds = model(batch['encodings'])
    
        all_embeddings_train.append(embeds)
        all_labels_train.extend(batch['labels'])
    
    all_embeddings_train = torch.cat(all_embeddings_train, dim=0)
    
    train_label_to_ids = defaultdict(list)
    for i, label in enumerate(all_labels_train):
        train_label_to_ids[label].append(i)
    
    all_labels_val = []
    all_embeddings_val = []
    for batch in dataloader_val:
        batch['encodings'] = {k : v.to(device) for k, v in batch['encodings'].items()}
    
        with torch.no_grad():
            embeds = model(batch['encodings'])
    
        all_embeddings_val.append(embeds)
        all_labels_val.extend(batch['labels'])
    
    all_embeddings_val = torch.cat(all_embeddings_val, dim=0)
    
    predicted = []
    for embed in all_embeddings_val:
        dists = criterion.calc_euclidean(embed, all_embeddings_train)
        min_label_dict = float('inf')
        pred = -1
        for label in train_label_to_ids.keys():
            ids = np.random.choice(train_label_to_ids[label], min(len(train_label_to_ids[label]), num_samples_to_evaluate), replace=False)
            label_dist = dists[ids].mean()
            if label_dist < min_label_dict:
                min_label_dict = label_dist
                pred = label
    
        predicted.append(pred)
    
    metric = f1_score(all_labels_val, predicted, average='weighted', zero_division=0.0)
    if metric > best_metric:
        best_metric = metric
        torch.save(model.state_dict(), 'weights/weights_few_shot.pt')
        
    print(f"F1 weighted: {metric}")
    
    print(classification_report(all_labels_val, predicted))
    print('='*50)

Training: 100%|████████████████████████████████| 18/18 [00:05<00:00,  3.47it/s]


Epoch: 1/8 - Training Loss: 0.3814


Epochs:  12%|████▌                               | 1/8 [00:06<00:47,  6.73s/it]

F1 weighted: 0.41981981981981986
                         precision    recall  f1-score   support

                   Jack       0.00      0.00      0.00         0
             SFP модуль       0.00      0.00      0.00         0
           Wi-fi модуль       0.00      0.00      0.00         1
            Аккумулятор       0.00      0.00      0.00         0
           Блок питания       1.00      0.25      0.40        12
               Динамики       0.00      0.00      0.00         0
                   Диск       0.75      0.75      0.75         4
                 Камера       0.00      0.00      0.00         1
                 Корпус       0.00      0.00      0.00         1
      Материнская плата       0.00      0.00      0.00         7
                Матрица       0.67      1.00      0.80         8
     Оперативная память       0.00      0.00      0.00         0
Программное обеспечение       0.00      0.00      0.00         1
                 Сервер       1.00      0.50      0.67  


Training: 100%|████████████████████████████████| 18/18 [00:05<00:00,  3.37it/s]


...

F1 weighted: 0.6571819598135388
                         precision    recall  f1-score   support

                   Jack       0.00      0.00      0.00         0
             SFP модуль       0.00      0.00      0.00         0
           Wi-fi модуль       1.00      1.00      1.00         1
            Аккумулятор       0.00      0.00      0.00         0
           Блок питания       1.00      0.83      0.91        12
             Вентилятор       0.00      0.00      0.00         0
                   Диск       1.00      0.75      0.86         4
                 Камера       0.00      0.00      0.00         1
                 Корпус       0.00      0.00      0.00         1
      Материнская плата       1.00      0.14      0.25         7
                Матрица       0.64      0.88      0.74         8
     Оперативная память       0.00      0.00      0.00         0
Программное обеспечение       0.00      0.00      0.00         1
                 Сервер       1.00      0.50      0.67   


Training: 100%|████████████████████████████████| 18/18 [00:05<00:00,  3.19it/s]


Epoch: 8/8 - Training Loss: 0.0375


Epochs: 100%|████████████████████████████████████| 8/8 [00:54<00:00,  6.84s/it]

F1 weighted: 0.5965758992074781
                         precision    recall  f1-score   support

                   Jack       0.00      0.00      0.00         0
           Wi-fi модуль       1.00      1.00      1.00         1
            Аккумулятор       0.00      0.00      0.00         0
           Блок питания       1.00      0.50      0.67        12
             Вентилятор       0.00      0.00      0.00         0
                   Диск       1.00      0.75      0.86         4
                 Камера       0.00      0.00      0.00         1
                 Корпус       0.00      0.00      0.00         1
      Материнская плата       1.00      0.14      0.25         7
                Матрица       0.64      0.88      0.74         8
     Оперативная память       0.00      0.00      0.00         0
Программное обеспечение       0.00      0.00      0.00         1
                 Сервер       1.00      1.00      1.00         2

               accuracy                           0.54  